# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследования ответить на вопросы:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Нам был представлен датасет с такими данными:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

План работы:
* изучить датасет на предмет наличия ошибок и артефактов
* исправить все найденные проблемы
* добавить недостающие, категоризовать имеющиеся вместе с новыми данные
* сделать наглядные подсчеты и сформулировать выводы

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [117]:
#импортируем библиотеку pandas, открываем файл по данному пути, смотрим информацию о датафрейме
#импортируем Mystem
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

children            21525 non-null int64

days_employed       19351 non-null float64

dob_years           21525 non-null int64

education           21525 non-null object

education_id        21525 non-null int64

family_status       21525 non-null object

family_status_id    21525 non-null int64

gender              21525 non-null object

income_type         21525 non-null object

debt                21525 non-null int64

total_income        19351 non-null float64

purpose             21525 non-null object

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


Видим, что есть пропуски, названия колонок адекватные и соответствуют данным в них. К типам данных вопросов нет пока что. 

In [118]:
#знакомимся с датафреймом более наглядно - выводим датафрейм методом display
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [119]:
#для начала переведем все отрицательные значения в положительные для удобства рассчетов
df['days_employed'] = abs(df['days_employed'])
df['children'] = abs(df['children'])

In [120]:
#в конце исследования я заметил, что есть люди, у которых 20 детей. Думаю, это ошибка, поэтому поменяю значение на 2
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод:**
<br>Здесь видим, 
<br>a) что есть отрицательные значения в колонке days_employed, а также неадекватно большие числа в той же колонке. Возможно, эти данные были записаны с ошибкой. Изучив, как рассчитывается трудовой стаж, можно сделать вывод, что некоторые (самые большие) значения записаны в часах.
<br>b) в колонке education значения представлены в разных регистрах. Это не сильно влияет на исследования, т.к. у нас есть колонка education_id, но мы выполняем проект по максимуму, поэтому приводим данные в этом столбце к нормальному виду.

## Шаг 2. Предобработка данных<a id="#preprocessing"></a>

### Обработка пропусков

In [121]:
#смотрим количество пропусков и в каких колонках
print(df.isna().sum())

children               0

days_employed       2174

dob_years              0

education              0

education_id           0

family_status          0

family_status_id       0

gender                 0

income_type            0

debt                   0

total_income        2174

purpose                0

dtype: int64


In [122]:
#Чтобы сразу нормально заполнить все пропуски, приведем явно выбивающиеся значения из колонки days_employed 
#к виду, который подразумевает моя теория об этих данных ниже, в выводе
df.loc[df['days_employed'] > 22000, 'days_employed'] = df.loc[df['days_employed'] > 22000, 'days_employed'] / 24
#Проверим, выполнилось ли условие
display((df['days_employed'] > 22000).sum())

0

In [142]:
#по этому датафрейму определим медиану для нормальных значений и сохраним ее в переменную days_employed_median
days_employed_median = df.groupby('income_type')['days_employed'].median()
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda s: s.fillna(s.mean()))
#заполним все пропуски в days_employed медианным значением
df['days_employed'] = df['days_employed'].fillna(median_days_employed.round(2))
#проверим, все ли сработало
print(df.isna().sum())

children            0

days_employed       0

dob_years           0

education           0

education_id        0

family_status       0

family_status_id    0

gender              0

income_type         0

debt                0

total_income        0

purpose             0

lemm_purpose        0

purpose_category    0

income_category     0

dtype: int64


Теперь по такому же принципу заполним пропуски в колонке total_income

In [124]:
#по этому датафрейму определим медиану для столбца total_income по группам работающих 
#и сохраним ее в переменной total_income_median
total_income_median = df.groupby('income_type')['total_income'].median()
#заполним все пропуски в total_income медианным значением
total_income_median = df.groupby('income_type')['total_income'].median()
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.mean()))
#проверим, все ли сработало
print(df.isna().sum())

children            0

days_employed       0

dob_years           0

education           0

education_id        0

family_status       0

family_status_id    0

gender              0

income_type         0

debt                0

total_income        0

purpose             0

dtype: int64


**Вывод**
<br>Почему могли появиться пропуски именно в этих колонках? Возможно, люди работали неофициально, стаж у них не записывался, доходы были "серыми". Заполнили пропуски медианными значениями.
<br>Чтобы адекватно заполнить медианным значением колонку days_employed, выяснили, все ли значения соответствуют формату записи. Как уже видно из первого вывода датафрейма, не все. Предположительно, такие большие значения - это тот же рабочий стаж, но выраженный в часах. Чтобы это проверить, отфильтровали значения по такому принципу - если значение больше 22000 (что соответствует ~61 году рабочего стажа, что случается только в очень редких случаях), то оно выражено в часах. 

### Замена типа данных

In [125]:
#смотрим, в каких колонках нам целесообразно и возможно изменить тип данных. 
#Видим, что можем изменить тип days_employed и total_income с float на int.
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

children            21525 non-null int64

days_employed       21525 non-null float64

dob_years           21525 non-null int64

education           21525 non-null object

education_id        21525 non-null int64

family_status       21525 non-null object

family_status_id    21525 non-null int64

gender              21525 non-null object

income_type         21525 non-null object

debt                21525 non-null int64

total_income        21525 non-null float64

purpose             21525 non-null object

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


In [126]:
#произведем замену
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
#проверим, все ли сработало
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

children            21525 non-null int64

days_employed       21525 non-null int64

dob_years           21525 non-null int64

education           21525 non-null object

education_id        21525 non-null int64

family_status       21525 non-null object

family_status_id    21525 non-null int64

gender              21525 non-null object

income_type         21525 non-null object

debt                21525 non-null int64

total_income        21525 non-null int64

purpose             21525 non-null object

dtypes: int64(7), object(5)

memory usage: 2.0+ MB


**Вывод**
<br>Мы сделали это для того, чтобы дальше pandas мог обработать все операции, которые мы хотим применить. Почему преобразование было совершено именно с использованием метода .astype()? Потому что нам необходимо привести значения колонок days_employed и total_income к типу int. 

### Обработка дубликатов

**Пояснение**
<br>Я сразу заметил разные регистры в колонке education, поэтому решил не томить с тем, чтобы выявить неявные дубликаты методом .value_counts(), который возвращает кол-во уникальных значений столбца, и привел все значения к одному региструю

In [127]:
#Проверим, все ли значения в колонке education записаны в одном регистре
print(df['education'].value_counts())

среднее                13750

высшее                  4718

СРЕДНЕЕ                  772

Среднее                  711

неоконченное высшее      668

ВЫСШЕЕ                   274

Высшее                   268

начальное                250

Неоконченное высшее       47

НЕОКОНЧЕННОЕ ВЫСШЕЕ       29

НАЧАЛЬНОЕ                 17

Начальное                 15

ученая степень             4

Ученая степень             1

УЧЕНАЯ СТЕПЕНЬ             1

Name: education, dtype: int64


In [128]:
#Приведем значения в этой колонке к нижнему регистру, чтобы в дальнейшем успешно удалить все дубликаты
df['education'] = df['education'].str.lower()
#И еще раз проверим, все ли значения в колонке education записаны в одном регистре
print(df['education'].value_counts())

среднее                15233

высшее                  5260

неоконченное высшее      744

начальное                282

ученая степень             6

Name: education, dtype: int64


**Пояснение**
<br>Меня еще в самом начале смутили данные, которые, на мой взгляд, выражены в часах в колонке days_employed, поэтому я решил привести их к нужному виду - стаж, выраженный в **днях**

In [129]:
#Также приведем явно выбивающиеся значения из колонки days_employed к виду, который подразумевает моя теория об этих данных выше
#df.loc[df['days_employed'] > 22000, 'days_employed'] = df.loc[df['days_employed'] > 22000, 'days_employed'] / 24
#Проверим, выполнилось ли условие
#display((df['days_employed'] > 22000).sum())

**Пояснение**
<br>Теперь можем выявить и удалить строки-дубликаты

In [130]:
#найдем количество дубликатов
print(df.duplicated().sum())
#удалим дубликаты
df = df.drop_duplicates().reset_index(drop=True)
#и проверим, все ли сработало
print(df.duplicated().sum())

71

0


**Вывод**
<br>Удалив строки-дубликаты, выводы по исследованию будут точнее. Для удаления дубликатов я использовал метод drop.duplicates, потому что с его помощью можно удалить сразу все строки-дубликаты. 
<br>На мой взгляд, дубликаты появились либо вследствие человеческой ошибки (человек подал данные два раза), либо вследствие ошибки системы сбора и формирования данных.

### Лемматизация

**Пояснение**
<br>На данном этапе необходимо провести лемматизацию столбца purpose, т.к. в нем содержатся данные о цели получения кредита. С помощью лемматизации на следующем этапе мы сможем выделить категории целей, что существенно облегчит нам работу и позволит получить более точные результаты исследования.

In [131]:
#с помощью функции lemma_purpose, которая возвращает нам лемму от значения колонки, 
#мы лемматизируем колонку purpose, а затем сохраняем новые значения в новой колонке lemm_purpose
#путем применения метода apply, аргументом которого стала как раз функция lemma_purpose, к столбцу purpose
def lemma_purpose(column):
    lemmas = m.lemmatize(column)
    return lemmas
df['lemm_purpose'] = df['purpose'].apply(lemma_purpose)

In [132]:
#затем по совету преподавателя мы собираем все леммы в один список
lemma_df = []
for i in df['lemm_purpose']:
    lemma_df.extend(i)

In [133]:
#и подсчитываем количество лемм методом Counter
from collections import Counter
print(Counter(lemma_df))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


**Вывод**
<br>Выделив леммы, будет совсем несложно выделить отдельные категории по словам. А это, в свою очередь, нужно для ответа на основные вопросы исследования.

### Категоризация данных

Чтобы ответить на основные вопросы данного исследования, необходимо привести данные к определенным категориям. Например, нужно понять:
* с какой целью люди берут кредит
* как различаются кредитуемые лица по количеству детей и как это влияет на выплату кредита в срок
* как люди различаются по уровню дохода и как это влияет на выплату кредита в срок
* как люди различаются по семейному положению и как это влияет на выплату кредита в срок

In [134]:
#С помощью функции set_purpose_category мы выделяем 5 категорий из колонки purpose
def set_purpose_category(lemma_purpose):
    if (('жилье' in lemma_purpose) or ('недвижимость' in lemma_purpose)) and ('строительство' not in lemma_purpose):
        return 'покупка недвижимости'
    if (('жилье' in lemma_purpose) or ('недвижимость' in lemma_purpose)) and ('строительство' in lemma_purpose):
        return 'строительство недвижимости'
    if 'автомобиль' in lemma_purpose:
        return 'покупка автомобиля'
    if 'образование' in lemma_purpose:
        return 'образование'
    if 'свадьба' in lemma_purpose:
        return 'свадьба'
    return lemma_purpose
#применяем метод .apply() к столбцу lemm_purpose и получаем новую колонку purpose_category, в которой
#каждой строке присваивается категория цели покупки в соответствии со столбцом purpose
df['purpose_category'] = df['lemm_purpose'].apply(set_purpose_category)
#проверим, как это сработало, методом .value_counts()
df['purpose_category'].value_counts()

покупка недвижимости          8933
покупка автомобиля            4306
образование                   4013
свадьба                       2324
строительство недвижимости    1878
Name: purpose_category, dtype: int64

In [135]:
#разбиваем колонку total_income на 4 равные части, чтобы выделить категории людей по доходам в месяц
pd.qcut(df['total_income'], 4)

0        (202417.0, 2265604.0]
1         (107623.0, 151887.0]
2         (107623.0, 151887.0]
3        (202417.0, 2265604.0]
4         (151887.0, 202417.0]
                 ...          
21449    (202417.0, 2265604.0]
21450     (151887.0, 202417.0]
21451    (20666.999, 107623.0]
21452    (202417.0, 2265604.0]
21453    (20666.999, 107623.0]
Name: total_income, Length: 21454, dtype: category
Categories (4, interval[float64]): [(20666.999, 107623.0] < (107623.0, 151887.0] < (151887.0, 202417.0] < (202417.0, 2265604.0]]

In [136]:
#а данной функцией присваиваем каждой строке определенную категорию в соответствии с уровнем дохода методом apply
#с добавлением новой колонки
def income_category(income):
    if 20666 <= income <= 107623.999:
        return 'low'
    if 107624 <= income <= 145017.999:
        return 'middle'
    if 145018 <= income <= 195813.999:
        return 'upper middle'
    if 195814 <= income <= 2265604:
        return 'high'
df['income_category'] = df['total_income'].apply(income_category)

**Вывод**
<br>Можно заметить, что категории некоторых видов данных (дети, образование) уже есть. Однако для исследования нам нужно выделить еще категории целей кредитования, а также категории людей по уровню дохода. Что мы и сделали в данном разделе.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [137]:
#Сгруппируем методом pivot_table колонки debt и children, чтобы определить, как влияет количество детей на выплату кредита
df_pivot_children = df.pivot_table(index = ['children'], columns = 'debt',
                                   values = 'family_status_id', aggfunc = 'count', fill_value=0)
#переименование столбцов для удобства чтения:
df_pivot_children.columns = ['no_debt', 'debtors']
#создадим еще 1 колонку, которая будет показывать процент должников по категориям
df_pivot_children['% debtors'] = df_pivot_children['debtors'] / (df_pivot_children['debtors'] + df_pivot_children['no_debt'])
display(df_pivot_children.style.format({('% debtors') : "{:.2%}"}))

,no_debt,debtors,% debtors
children,,,
0,13028,1063,7.54%
1,4410,445,9.17%
2,1926,202,9.49%
3,303,27,8.18%
4,37,4,9.76%
5,9,0,0.00%


**Вывод**

Как видим, люди, у которых нет детей, реже имеют проблемы с оплатой кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [138]:
#Сгруппируем методом pivot_table колонки debt и family_status, чтобы определить, 
#как влияет семейное положение на выплату кредита
df_pivot_family_status = df.pivot_table(index = ['family_status'], columns = 'debt', 
                                        values = 'children', aggfunc = 'count', fill_value=0)
#переименование столбцов для удобства чтения:
df_pivot_family_status.columns = ['no_debt', 'debtors']
#создадим еще 1 колонку, которая будет показывать процент должников по категориям
df_pivot_family_status['% debtors'] = df_pivot_family_status['debtors'] / (df_pivot_family_status['debtors'] + 
                                                                           df_pivot_family_status['no_debt'])
display(df_pivot_family_status.style.format({('% debtors') : "{:.2%}"}))

,no_debt,debtors,% debtors
family_status,,,
Не женат / не замужем,2536,274,9.75%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%


**Вывод**

Видно, что люди, которые в статусе "не женат/не замужем" и "гражданский брак", чаще остальных имеют проблемы с оплатой обязательств по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [139]:
#Сгруппируем методом pivot_table колонки debt и income_category, чтобы определить, как влияет уровень дохода на выплату кредита
df_pivot_income = df.pivot_table(index = ['income_category'], columns = 'debt', 
                                 values = 'children', aggfunc = 'count', fill_value=0)
#переименование столбцов для удобства чтения:
df_pivot_income.columns = ['no_debt', 'debtors']
#создадим еще 1 колонку, которая будет показывать процент должников по категориям
df_pivot_income['% debtors'] = df_pivot_income['debtors'] / (df_pivot_income['debtors'] + 
                                                                           df_pivot_income['no_debt'])
display(df_pivot_income.sort_values(by='income_category').style.format({('% debtors') : "{:.2%}"}))

,no_debt,debtors,% debtors
income_category,,,
high,5453,413,7.04%
low,4937,427,7.96%
middle,4286,412,8.77%
upper middle,5037,489,8.85%


**Вывод**

Можно отметить, что люди с низким и высоким уровнем дохода реже имеют проблемы с оплатой обязательств по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [140]:
#Сгруппируем методом pivot_table колонки debt и purpose_category, чтобы определить, как влияет цель кредита на выплату кредита
df_pivot_purpose = df.pivot_table(index = ['purpose_category'], columns = 'debt', 
                                  values = 'children', aggfunc = 'count', fill_value=0)
#переименование столбцов для удобства чтения:
df_pivot_purpose.columns = ['no_debt', 'debtors']
#создадим еще 1 колонку, которая будет показывать процент должников по категориям
df_pivot_purpose['% debtors'] = df_pivot_purpose['debtors'] / (df_pivot_purpose['debtors'] + 
                                                                           df_pivot_purpose['no_debt'])
display(df_pivot_purpose.style.format({('% debtors') : "{:.2%}"}))

,no_debt,debtors,% debtors
purpose_category,,,
образование,3643,370,9.22%
покупка автомобиля,3903,403,9.36%
покупка недвижимости,8295,638,7.14%
свадьба,2138,186,8.00%
строительство недвижимости,1734,144,7.67%


**Вывод**

Люди, которые берут кредит на покупку или строительство недвижимости, реже остальных имеют проблемы с оплатой обязательств по кредиту.

## Шаг 4. Общий вывод

Данный нам для исследования датасет информативен, хоть и содержал определенные ошибки. Благодаря выполненным выше действиям удалось выяснить, какие группы людей могут принести проблемы банку, а какие группы являются более надежными. Из этого исследования следует, что самые надежные заемщики это люди, у которых:
* нет детей
* семейное положение - женат/замужем, вдовец/вдова, в разводе
* у которых доход низкий или высокий
* цель кредитования которых приобретение или строительство недвижимости

Таким образом, цели исследования была достигнута, банк доволен, люди не очень.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.